In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from reportlab.lib.pagesizes import letter
from reportlab.lib import colors
from reportlab.lib.units import inch 
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Image, Table, TableStyle, PageBreak
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.lib.enums import TA_JUSTIFY #Para justificar el texto
from reportlab.lib.styles import ParagraphStyle #Para justificar el texto, tipo de letra
from reportlab.lib.enums import TA_CENTER #Para centrar títulos o texto
import geopandas as gpd
import plotly.express as px
import plotly.io as pio

import numpy as np
import plotly.graph_objects as go


LECTURA DE LOS DATAFRAMES

In [30]:
df_python = pd.read_pickle('C:/Users/USUARIO/OneDrive - Departamento Nacional de Planeacion/maalarcon_files/0. Resources/1. Bases MDN/2. Febrero 2024/outputs/240226_BaseMDN_Febrero24.plk')

In [31]:
# Crear la Columna de Código de Departamento
df_python['CODIGO_DPTO'] = df_python['COD_MUNI'].str[:2]

- (Opcional) Filtrar Delitos y Años

In [4]:
replacements = {
    'HOMICIDIO': 'homicidio',
    'HURTO A PERSONAS': 'hurto a personas',
    'HURTO COMERCIO': 'hurto a comercio',
    'HURTO RESIDENCIAS': 'hurto a residencias',
    'HURTO VEHICULOS': 'hurto a vehículos',
    'LESIONES PERSONALES': 'lesiones personales',
    'DELITOS SEXUALES': 'delitos sexuales',
    'VIOLENCIA INTRAFAMILIAR': 'violencia intrafamiliar',
    'EXTORSION': 'extorsión',
    'TERRORISMO': 'terrorismo'
}

df_python['DELITO'] = df_python['DELITO'].replace(replacements)

In [ ]:
# Lista de valores para conservar
delitos_filter = ['HOMICIDIO', 'HURTO A PERSONAS', 'LESIONES PERSONALES', 'SECUESTRO', 'VIOLENCIA INTRAFAMILIAR', 'EXTORSION']

# Filtra el DataFrame
df_filtrado = df_python[df_python['delito'].isin(delitos_filter)]

df_python = df_filtrado

df_python['anio']=df_python['anio'].astype(int)

años_filter = [2017,2018,2019,2020,2021,2022,2023]

df_python = df_python[df_python['anio'].isin(años_filter)]

- Lectura bases de población

In [5]:
#df_factores_riesgo = pd.read_pickle('C:/Users/USUARIO/OneDrive - Departamento Nacional de Planeacion/maalarcon_files/DNP-1389-2023/2023 08 31 - Stx Reportes Analytics/tmp/data_factores_riesgo.pkl')
df_pob_sexo = pd.read_pickle('C:/Users/USUARIO/OneDrive - Departamento Nacional de Planeacion/maalarcon_files/DNP-1389-2023/2023 08 31 - Stx Reportes Analytics/tmp/df_pob_sexo.pkl')

In [6]:
df_poblacion_a = pd.read_excel('C:/Users/USUARIO/OneDrive - Departamento Nacional de Planeacion/maalarcon_files/0. Resources/Proyecciones Población DANE/Sexo/DCD-area-sexo-edad-proypoblacion-Mun-1995-2004.xlsx')
df_poblacion_b = pd.read_excel('C:/Users/USUARIO/OneDrive - Departamento Nacional de Planeacion/maalarcon_files/0. Resources/Proyecciones Población DANE/Sexo/DCD-area-sexo-edad-proypoblacion-Mun-2005-2019.xlsx')
df_poblacion_c = pd.read_excel('C:/Users/USUARIO/OneDrive - Departamento Nacional de Planeacion/maalarcon_files/0. Resources/Proyecciones Población DANE/Sexo/DCD-area-sexo-edad-proypoblacion-Mun-2020-2035-ActPostCOVID-19.xlsx')

df_poblacion_d = df_poblacion_a.merge(df_poblacion_b, how='outer')
df_poblacion = df_poblacion_d.merge(df_poblacion_c, how='outer')

df_poblacion['CODIGO_DANE'] = df_poblacion['CODIGO_DANE'].apply(lambda x: f"0{x}" if len(str(x)) == 4 else x)
df_poblacion['CODIGO_DPTO'] = df_poblacion['CODIGO_DPTO'].apply(lambda x: f"0{x}" if len(str(x)) == 1 else x)

df_poblacion['CODIGO_DANE'] = df_poblacion['CODIGO_DANE'].astype(str)
df_poblacion['AÑO'] = df_poblacion['AÑO'].astype('int64')

In [7]:
df_pob_area = pd.read_excel('C:/Users/USUARIO/OneDrive - Departamento Nacional de Planeacion/maalarcon_files/0. Resources/Proyecciones Población DANE/Zona (área)/DCD-area-proypoblacion-Mun-1995-2035.xlsx')

df_pob_area['COD_MUNI'] = df_pob_area['COD_MUNI'].astype(str)
df_pob_area['CODIGO_DANE'] = df_pob_area['COD_MUNI'].apply(lambda x: f"0{x}" if len(str(x)) == 4 else x)
df_pob_area['COD_MUNI'] = df_pob_area['COD_MUNI'].apply(lambda x: f"0{x}" if len(str(x)) == 4 else x)

df_pob_area['CODIGO_DANE'] = df_pob_area['CODIGO_DANE'].astype(str)

# Creación columna CODIGO_DPTO
df_pob_area['CODIGO_DPTO'] = df_pob_area['CODIGO_DANE'].str[:2]
df_pob_area['CODIGO_DPTO'] = df_pob_area['CODIGO_DPTO'].astype(str)

TABLAS

Tablas Nacional, Departamentos y Municipios

In [32]:
poblacion_nacional = df_poblacion.groupby('AÑO')['POBLACION'].sum().reset_index()
poblacion_departamento = df_poblacion.groupby(['AÑO', 'CODIGO_DPTO','DPTO'])['POBLACION'].sum().reset_index()

In [33]:
casos_nacional = df_python.groupby(['DELITO', 'AÑO'])['CANTIDAD'].sum().reset_index()#Check
casos_departamento = df_python.groupby(['CODIGO_DPTO','DELITO', 'AÑO'])['CANTIDAD'].sum().reset_index()
casos_municipio = df_python.groupby(['COD_MUNI','DELITO','AÑO'])['CANTIDAD'].sum().reset_index()

In [35]:
casos_departamento['CODIGO_DPTO'] =casos_departamento['CODIGO_DPTO'].astype(str)
casos_departamento['AÑO'] =casos_departamento['AÑO'].astype(int) 


poblacion_departamento['CODIGO_DPTO'] =poblacion_departamento['CODIGO_DPTO'].astype(str)
poblacion_departamento['AÑO'] =poblacion_departamento['AÑO'].astype(int) 

casos_municipio['COD_MUNI'] =casos_municipio['COD_MUNI'].astype(str)
casos_municipio['AÑO'] =casos_municipio['AÑO'].astype('int64') 


In [36]:
casos_nacional = casos_nacional.merge(poblacion_nacional, left_on='AÑO', right_on='AÑO', how='left')
casos_departamento = casos_departamento.merge(poblacion_departamento, left_on=['CODIGO_DPTO','AÑO'], right_on=['CODIGO_DPTO','AÑO'], how='left')
casos_municipio = casos_municipio.merge(df_poblacion[['CODIGO_DPTO', 'DPTO', 'CODIGO_DANE', 'MPIO', 'AÑO','POBLACION']], left_on=['COD_MUNI','AÑO'], right_on=['CODIGO_DANE','AÑO'], how='left')

In [37]:
casos_nacional['TASA_NAL'] = (casos_nacional['CANTIDAD'] / casos_nacional['POBLACION'] ) * 100000
casos_departamento['TASA_DPTO'] = (casos_departamento['CANTIDAD'] / casos_departamento['POBLACION']) * 100000
casos_municipio['TASA_MPIO'] = (casos_municipio['CANTIDAD'] / casos_municipio['POBLACION']) * 100000

casos_nacional['TASA_NAL'] = casos_nacional['TASA_NAL'].round(1)
casos_departamento['TASA_DPTO'] = casos_departamento['TASA_DPTO'].round(1)
casos_municipio['TASA_MPIO'] = casos_municipio['TASA_MPIO'].round(1)

In [38]:
# Ordenar los datos para asegurar que la variación se calcula correctamente
casos_nacional = casos_nacional.sort_values(by=['DELITO', 'AÑO'])

# Calcular la variación porcentual de la cantidad de delitos
casos_nacional['VAR'] = casos_nacional.groupby(['DELITO'])['CANTIDAD'].pct_change() * 100
casos_nacional['VAR'] = casos_nacional['VAR'].round(1)
casos_nacional['VAR'] = casos_nacional['VAR'].fillna(0)

#------------------------------------------------------------------------------------------------------------------------

# Ordenar los datos para asegurar que la variación se calcula correctamente
casos_departamento = casos_departamento.sort_values(by=['CODIGO_DPTO', 'DELITO', 'AÑO'])

# Calcular la variación porcentual de la cantidad de delitos
casos_departamento['VAR'] = casos_departamento.groupby(['CODIGO_DPTO', 'DELITO'])['CANTIDAD'].pct_change() * 100
casos_departamento['VAR'] = casos_departamento['VAR'].round(1)
casos_departamento['VAR'] = casos_departamento['VAR'].fillna(0)

#------------------------------------------------------------------------------------------------------------------------

# Ordenar los datos para asegurar que la variación se calcula correctamente
casos_municipio = casos_municipio.sort_values(by=['COD_MUNI', 'DELITO', 'AÑO'])

# Calcular la variación porcentual de la cantidad de delitos
casos_municipio['VAR'] = casos_municipio.groupby(['COD_MUNI', 'DELITO'])['CANTIDAD'].pct_change() * 100
casos_municipio['VAR'] = casos_municipio['VAR'].round(1)
casos_municipio['VAR'] = casos_municipio['VAR'].fillna(0)



Tablas Nacional, Departamentos y Municipios (Meses)

In [39]:
casos_nacional_mes = df_python.groupby(['DELITO', 'AÑO', 'MES'])['CANTIDAD'].sum().reset_index()#Check
casos_departamento_mes = df_python.groupby(['CODIGO_DPTO','DELITO', 'AÑO', 'MES'])['CANTIDAD'].sum().reset_index()
casos_municipio_mes = df_python.groupby(['COD_MUNI','DELITO','AÑO', 'MES'])['CANTIDAD'].sum().reset_index()

In [40]:
casos_departamento_mes['CODIGO_DPTO'] =casos_departamento_mes['CODIGO_DPTO'].astype(str)
casos_departamento_mes['AÑO'] =casos_departamento_mes['AÑO'].astype(int) 


poblacion_departamento['CODIGO_DPTO'] =poblacion_departamento['CODIGO_DPTO'].astype(str)
poblacion_departamento['AÑO'] =poblacion_departamento['AÑO'].astype(int) 

casos_municipio_mes['COD_MUNI'] =casos_municipio_mes['COD_MUNI'].astype(str)
casos_municipio_mes['AÑO'] =casos_municipio_mes['AÑO'].astype('int64') 

In [41]:
casos_nacional_mes = casos_nacional_mes.merge(poblacion_nacional, left_on='AÑO', right_on='AÑO', how='left')
casos_departamento_mes = casos_departamento_mes.merge(poblacion_departamento, left_on=['CODIGO_DPTO','AÑO'], right_on=['CODIGO_DPTO','AÑO'], how='left')
casos_municipio_mes = casos_municipio_mes.merge(df_poblacion[['CODIGO_DPTO', 'DPTO', 'CODIGO_DANE', 'MPIO', 'AÑO','POBLACION']], left_on=['COD_MUNI','AÑO'], right_on=['CODIGO_DANE','AÑO'], how='left')

In [42]:
casos_nacional_mes['TASA_NAL'] = (casos_nacional_mes['CANTIDAD'] / casos_nacional_mes['POBLACION'] ) * 100000
casos_departamento_mes['TASA_DPTO'] = (casos_departamento_mes['CANTIDAD'] / casos_departamento_mes['POBLACION']) * 100000
casos_municipio_mes['TASA_MPIO'] = (casos_municipio_mes['CANTIDAD'] / casos_municipio_mes['POBLACION']) * 100000

casos_nacional_mes['TASA_NAL'] = casos_nacional_mes['TASA_NAL'].round(1)
casos_departamento_mes['TASA_DPTO'] = casos_departamento_mes['TASA_DPTO'].round(1)
casos_municipio_mes['TASA_MPIO'] = casos_municipio_mes['TASA_MPIO'].round(1)

In [43]:
# Ordenar los datos para asegurar que la variación se calcula correctamente
casos_nacional_mes = casos_nacional_mes.sort_values(by=['DELITO', 'AÑO'])

# Calcular la variación porcentual de la cantidad de delitos
casos_nacional_mes['VAR'] = casos_nacional_mes.groupby(['DELITO','AÑO'])['CANTIDAD'].pct_change() * 100
casos_nacional_mes['VAR'] = casos_nacional_mes['VAR'].round(1)
casos_nacional_mes['VAR'] = casos_nacional_mes['VAR'].fillna(0)

#------------------------------------------------------------------------------------------------------------------------

# Ordenar los datos para asegurar que la variación se calcula correctamente
casos_departamento_mes = casos_departamento_mes.sort_values(by=['CODIGO_DPTO', 'DELITO', 'AÑO'])

# Calcular la variación porcentual de la cantidad de delitos
casos_departamento_mes['VAR'] = casos_departamento_mes.groupby(['CODIGO_DPTO', 'DELITO','AÑO'])['CANTIDAD'].pct_change() * 100
casos_departamento_mes['VAR'] = casos_departamento_mes['VAR'].round(1)
casos_departamento_mes['VAR'] = casos_departamento_mes['VAR'].fillna(0)

#------------------------------------------------------------------------------------------------------------------------

# Ordenar los datos para asegurar que la variación se calcula correctamente
casos_municipio_mes = casos_municipio_mes.sort_values(by=['COD_MUNI', 'DELITO', 'AÑO'])

# Calcular la variación porcentual de la cantidad de delitos
casos_municipio_mes['VAR'] = casos_municipio_mes.groupby(['COD_MUNI', 'DELITO','AÑO'])['CANTIDAD'].pct_change() * 100
casos_municipio_mes['VAR'] = casos_municipio_mes['VAR'].round(1)
casos_municipio_mes['VAR'] = casos_municipio_mes['VAR'].fillna(0)

Tablas Variación por periodos de tiempo

In [68]:
codigo_dane = '73001'
codigo_dane = str(codigo_dane)


min_mes = 1
max_mes = 2

In [45]:
def suma_delitos_periodo(data, inicio_mes, fin_mes, codigo_dane):
    """
    Esta función calcula la suma de la variable 'CANTIDAD' para cada 'DELITO' dentro de un periodo específico de cada año.
    
    Parámetros:
    - data: DataFrame que contiene los datos.
    - inicio_mes: El mes de inicio del periodo (1 = enero, 2 = febrero, ..., 12 = diciembre).
    - fin_mes: El mes de fin del periodo (1 = enero, 2 = febrero, ..., 12 = diciembre).
    
    Retorna:
    - DataFrame con la suma de 'CANTIDAD' para cada 'DELITO' agrupado por 'DELITO' y 'AÑO'.
    """
    
    # Filtrar el DataFrame por el periodo de meses especificado
    periodo_filtrado = data[(data['MES'] >= inicio_mes) & (data['MES'] <= fin_mes) & (data['COD_MUNI'] == codigo_dane)]
    
    # Agrupar por 'DELITO' y 'AÑO', y sumar 'CANTIDAD'
    suma_por_delito = periodo_filtrado.groupby(['DELITO', 'AÑO','DPTO', 'MPIO'])['CANTIDAD'].sum().reset_index()

    # Calcular la variación porcentual de 'CANTIDAD' entre años para cada 'DELITO'
    suma_por_delito['VAR%'] = suma_por_delito.groupby('DELITO')['CANTIDAD'].pct_change() * 100

    suma_por_delito['VAR%'] = suma_por_delito['VAR%'].round(1)

    suma_por_delito['VAR%'] = suma_por_delito['VAR%'].fillna(0)
    
    return suma_por_delito

In [46]:
# Ejemplo de uso de la función para el periodo de enero a abril
resultado_periodo = suma_delitos_periodo(casos_municipio_mes, min_mes, max_mes, codigo_dane)
resultado_periodo.head()

resultado_periodo['CANTIDAD'] =resultado_periodo['CANTIDAD'].astype(int) 

resultado_periodo.head()

,DELITO,AÑO,DPTO,MPIO,CANTIDAD,VAR%
0,ACCIONES SUBVERSIVAS,2004,Tolima,Ibagué,1,0.0
1,AFECTACION FUERZA PUBLICA,2010,Tolima,Ibagué,4,0.0
2,AFECTACION FUERZA PUBLICA,2011,Tolima,Ibagué,7,75.0
3,AFECTACION FUERZA PUBLICA,2012,Tolima,Ibagué,1,-85.7
4,AFECTACION FUERZA PUBLICA,2013,Tolima,Ibagué,4,300.0


Exportar tablas a excel

In [47]:
# casos_nacional.to_excel('C:/Users/USUARIO/OneDrive - Departamento Nacional de Planeacion/maalarcon_files/0. Resources/1. Bases MDN/1. Enero 2024/outputs/resultados_nacional.xlsx')
# casos_departamento.to_excel('C:/Users/USUARIO/OneDrive - Departamento Nacional de Planeacion/maalarcon_files/0. Resources/1. Bases MDN/1. Enero 2024/outputs/resultados_departamento.xlsx')
# casos_municipio.to_excel('C:/Users/USUARIO/OneDrive - Departamento Nacional de Planeacion/maalarcon_files/0. Resources/1. Bases MDN/1. Enero 2024/outputs/resultados_municipio.xlsx')

In [48]:
# casos_nacional_mes.to_csv('C:/Users/USUARIO/OneDrive - Departamento Nacional de Planeacion/maalarcon_files/0. Resources/1. Bases MDN/1. Enero 2024/outputs/resultados_nacional_mes.csv')
# casos_departamento_mes.to_csv('C:/Users/USUARIO/OneDrive - Departamento Nacional de Planeacion/maalarcon_files/0. Resources/1. Bases MDN/1. Enero 2024/outputs/resultados_departamento_mes.csv')
# casos_municipio_mes.to_csv('C:/Users/USUARIO/OneDrive - Departamento Nacional de Planeacion/maalarcon_files/0. Resources/1. Bases MDN/1. Enero 2024/outputs/resultados_municipio_mes.csv')

Documento en word


In [49]:
# Datos para el DataFrame
meses_año = {
    'Número de Mes': range(1, 13),
    'Nombre del Mes': [
        'Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio', 
        'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre'
    ]
}
# Crear el DataFrame
df_meses_año = pd.DataFrame(meses_año)

In [69]:
nombre_mpio = resultado_periodo['MPIO'].loc[1]
nombre_dpto = resultado_periodo['DPTO'].loc[1]

años_comparacion_completo = [2022,2023]
años_comparacion_completo = [int(año) for año in años_comparacion_completo]

años_comparacion_periodo = [2023,2024]
años_comparacion_completo = [int(año) for año in años_comparacion_completo]

años_penultimo = [2022]
años_ultimo = [2023]

min_mes_nombre = df_meses_año[df_meses_año['Número de Mes']==min_mes]
min_mes_nombre = min_mes_nombre['Nombre del Mes'].iloc[0]

max_mes_nombre = df_meses_año[df_meses_año['Número de Mes']==max_mes]
max_mes_nombre = max_mes_nombre['Nombre del Mes'].iloc[0]

# Acceder al primer elemento (2022)
primer_año = años_comparacion_completo[0]

# Acceder al segundo elemento (2023)
segundo_año = años_comparacion_completo[1]


# Acceder al primer elemento (2022)
primer_año_periodo = años_comparacion_periodo[0]

# Acceder al segundo elemento (2023)
segundo_año_periodo = años_comparacion_periodo[1]



In [70]:
resultado_periodo['DELITO'].unique()

array(['ACCIONES SUBVERSIVAS', 'AFECTACION FUERZA PUBLICA',
       'CAPTURAS MINERIA ILEGAL', 'DELITOS CONTRA MEDIO AMBIENTE',
       'DELITOS SEXUALES', 'EXTORSION', 'HOMICIDIO',
       'HOMICIDIO ACCIDENTE TRANSITO', 'HURTO A PERSONAS',
       'HURTO COMERCIO', 'HURTO ENTIDADES FINANCIERAS',
       'HURTO RESIDENCIAS', 'HURTO VEHICULOS', 'INCAUTACION COCAINA',
       'INCAUTACION HEROINA', 'INCAUTACION MARIHUANA',
       'LESIONES ACCIDENTES TRANSITO', 'LESIONES PERSONALES',
       'MINAS INTERVENIDAS', 'SECUESTRO', 'TERRORISMO',
       'VIOLENCIA INTRAFAMILIAR', 'VOLADURA PUENTES VIAS'], dtype=object)

In [71]:
casos_municipio = casos_municipio.rename(columns={'VAR':'VAR%'})

In [90]:
años_test = [2022]
años_test = [int(año) for año in años_test]

In [91]:
delitos_libertad_secu_total = casos_municipio[(casos_municipio['DELITO']=='SECUESTRO') & (resultado_periodo['AÑO']==años_test) & (casos_municipio['COD_MUNI']==codigo_dane)]

ValueError: ('Lengths must match to compare', (339,), (1,))

Resultados año completo en word

In [72]:
#----------------delitos contra la vida
delitos_vida_hom_total = casos_municipio[(casos_municipio['DELITO']=='HOMICIDIO') & (casos_municipio['AÑO'].isin(años_comparacion_completo)) & (casos_municipio['COD_MUNI']==codigo_dane)]
delitos_vida_lper_total = casos_municipio[(casos_municipio['DELITO']=='LESIONES PERSONALES') & (casos_municipio['AÑO'].isin(años_comparacion_completo)) & (casos_municipio['COD_MUNI']==codigo_dane)]

#----------------último

delitos_libertad_secu_total = casos_municipio[(casos_municipio['DELITO']=='SECUESTRO') & (resultado_periodo['AÑO'].isin(años_comparacion_completo)) & (casos_municipio['COD_MUNI']==codigo_dane)]

#----------------último

delitos_patrimonio_hper_total = casos_municipio[(casos_municipio['DELITO']=='HURTO A PERSONAS') & (casos_municipio['AÑO'].isin(años_comparacion_completo)) & (casos_municipio['COD_MUNI']==codigo_dane)]
delitos_patrimonio_hcom_total = casos_municipio[(casos_municipio['DELITO']=='HURTO COMERCIO') & (casos_municipio['AÑO'].isin(años_comparacion_completo)) & (casos_municipio['COD_MUNI']==codigo_dane)]
delitos_patrimonio_hres_total = casos_municipio[(casos_municipio['DELITO']=='HURTO RESIDENCIAS') & (casos_municipio['AÑO'].isin(años_comparacion_completo)) & (casos_municipio['COD_MUNI']==codigo_dane)]
delitos_patrimonio_hveh_total = casos_municipio[(casos_municipio['DELITO']=='HURTO VEHICULOS') & (casos_municipio['AÑO'].isin(años_comparacion_completo)) & (casos_municipio['COD_MUNI']==codigo_dane)]
delitos_patrimonio_exto_total = casos_municipio[(casos_municipio['DELITO']=='EXTORSION') & (casos_municipio['AÑO'].isin(años_comparacion_completo)) & (casos_municipio['COD_MUNI']==codigo_dane)]


#----------------último

delitos_familia_vif_total = casos_municipio[(casos_municipio['DELITO']=='VIOLENCIA INTRAFAMILIAR') & (casos_municipio['AÑO'].isin(años_comparacion_completo)) & (casos_municipio['COD_MUNI']==codigo_dane)]

#----------------último
delitos_seg_publica_terr_total = casos_municipio[(casos_municipio['DELITO']=='TERRORISMO') & (casos_municipio['AÑO'].isin(años_comparacion_completo)) & (casos_municipio['COD_MUNI']==codigo_dane)]

# Se debe convertir la columna para que en el word no aparezcan los valores con un cero al final
delitos_vida_hom_total['CANTIDAD'] = delitos_vida_hom_total['CANTIDAD'].astype(int)
delitos_vida_lper_total['CANTIDAD'] = delitos_vida_lper_total['CANTIDAD'].astype(int)
delitos_libertad_secu_total['CANTIDAD'] = delitos_libertad_secu_total['CANTIDAD'].astype(int)
delitos_patrimonio_hper_total['CANTIDAD'] = delitos_patrimonio_hper_total['CANTIDAD'].astype(int)
delitos_patrimonio_hcom_total['CANTIDAD'] = delitos_patrimonio_hcom_total['CANTIDAD'].astype(int)
delitos_patrimonio_hres_total['CANTIDAD'] = delitos_patrimonio_hres_total['CANTIDAD'].astype(int)
delitos_patrimonio_hveh_total['CANTIDAD'] = delitos_patrimonio_hveh_total['CANTIDAD'].astype(int)
delitos_patrimonio_exto_total['CANTIDAD'] = delitos_patrimonio_exto_total['CANTIDAD'].astype(int)
delitos_familia_vif_total['CANTIDAD'] = delitos_familia_vif_total['CANTIDAD'].astype(int)
delitos_seg_publica_terr_total['CANTIDAD'] = delitos_seg_publica_terr_total['CANTIDAD'].astype(int)


delitos_vida_hom_total['DELITO'] = delitos_vida_hom_total['DELITO'].replace({'HOMICIDIO': 'homicidio'})
delitos_vida_lper_total['DELITO'] = delitos_vida_lper_total['DELITO'].replace({'LESIONES PERSONALES': 'lesiones personales'})
delitos_libertad_secu_total['DELITO'] = delitos_libertad_secu_total['DELITO'].replace({'SECUESTRO': 'Secuestro'})
delitos_patrimonio_hper_total['DELITO'] = delitos_patrimonio_hper_total['DELITO'].replace({'HURTO A PERSONAS': 'hurto a personas'})
delitos_patrimonio_hcom_total['DELITO'] = delitos_patrimonio_hcom_total['DELITO'].replace({'HURTO COMERCIO': 'hurto a comercio'})
delitos_patrimonio_hres_total['DELITO'] = delitos_patrimonio_hres_total['DELITO'].replace({'HURTO RESIDENCIAS': 'hurto a residencias'})
delitos_patrimonio_hveh_total['DELITO'] = delitos_patrimonio_hveh_total['DELITO'].replace({'HURTO VEHICULOS': 'hurto a vehículos'})
delitos_patrimonio_exto_total['DELITO'] = delitos_patrimonio_exto_total['DELITO'].replace({'EXTORSION': 'extorsión'})
delitos_familia_vif_total['DELITO'] = delitos_familia_vif_total['DELITO'].replace({'VIOLENCIA INTRAFAMILIAR': 'violencia intrafamiliar'})
delitos_seg_publica_terr_total['DELITO'] = delitos_seg_publica_terr_total['DELITO'].replace({'TERRORISMO': 'terrorismo'})

C:\Users\USUARIO\AppData\Local\Temp\ipykernel_14816\1351213596.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  delitos_libertad_secu_total = casos_municipio[(casos_municipio['DELITO']=='SECUESTRO') & (resultado_periodo['AÑO'].isin(años_comparacion_completo)) & (casos_municipio['COD_MUNI']==codigo_dane)]
C:\Users\USUARIO\AppData\Local\Temp\ipykernel_14816\1351213596.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  delitos_vida_hom_total['CANTIDAD'] = delitos_vida_hom_total['CANTIDAD'].astype(int)
C:\Users\USUARIO\AppData\Local\Temp\ipykernel_14816\1351213596.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inste

Resultados periodo de año en word

In [ ]:
#----------------delitos contra la vida
delitos_vida_hom = resultado_periodo[(resultado_periodo['DELITO']=='HOMICIDIO') & (resultado_periodo['AÑO'].isin(años_comparacion_periodo))]
delitos_vida_lper= resultado_periodo[(resultado_periodo['DELITO']=='LESIONES PERSONALES') & (resultado_periodo['AÑO'].isin(años_comparacion_periodo))]

#----------------delitos contra la libertad

delitos_libertad_secu = resultado_periodo[(resultado_periodo['DELITO']=='SECUESTRO') & (resultado_periodo['AÑO'].isin(años_comparacion_periodo))]

#----------------delitos contra el patrimonio

delitos_patrimonio_hper = resultado_periodo[(resultado_periodo['DELITO']=='HURTO A PERSONAS') & (resultado_periodo['AÑO'].isin(años_comparacion_periodo))]
delitos_patrimonio_hcom = resultado_periodo[(resultado_periodo['DELITO']=='HURTO COMERCIO') & (resultado_periodo['AÑO'].isin(años_comparacion_periodo))]
delitos_patrimonio_hres = resultado_periodo[(resultado_periodo['DELITO']=='HURTO RESIDENCIAS') & (resultado_periodo['AÑO'].isin(años_comparacion_periodo))]
delitos_patrimonio_hveh = resultado_periodo[(resultado_periodo['DELITO']=='HURTO VEHICULOS') & (resultado_periodo['AÑO'].isin(años_comparacion_periodo))]
delitos_patrimonio_exto = resultado_periodo[(resultado_periodo['DELITO']=='EXTORSION') & (resultado_periodo['AÑO'].isin(años_comparacion_periodo))]


#----------------delitos contra la familia

delitos_familia_vif = resultado_periodo[(resultado_periodo['DELITO']=='VIOLENCIA INTRAFAMILIAR') & (resultado_periodo['AÑO'].isin(años_comparacion_periodo))]

#----------------delitos contra la seguridad publica
delitos_seg_publica_terr = resultado_periodo[(resultado_periodo['DELITO']=='TERRORISMO') & (resultado_periodo['AÑO'].isin(años_comparacion_periodo))]



# Se debe convertir la columna para que en el word no aparezcan los valores con un cero al final
delitos_vida_hom['CANTIDAD'] = delitos_vida_hom['CANTIDAD'].astype(int)
delitos_vida_lper['CANTIDAD'] = delitos_vida_lper['CANTIDAD'].astype(int)
delitos_libertad_secu['CANTIDAD'] = delitos_libertad_secu['CANTIDAD'].astype(int)
delitos_patrimonio_hper['CANTIDAD'] = delitos_patrimonio_hper['CANTIDAD'].astype(int)
delitos_patrimonio_hcom['CANTIDAD'] = delitos_patrimonio_hcom['CANTIDAD'].astype(int)
delitos_patrimonio_hres['CANTIDAD'] = delitos_patrimonio_hres['CANTIDAD'].astype(int)
delitos_patrimonio_hveh['CANTIDAD'] = delitos_patrimonio_hveh['CANTIDAD'].astype(int)
delitos_patrimonio_exto['CANTIDAD'] = delitos_patrimonio_exto['CANTIDAD'].astype(int)
delitos_familia_vif['CANTIDAD'] = delitos_familia_vif['CANTIDAD'].astype(int)
delitos_seg_publica_terr['CANTIDAD'] = delitos_seg_publica_terr['CANTIDAD'].astype(int)


delitos_vida_hom['DELITO'] = delitos_vida_hom['DELITO'].replace({'HOMICIDIO': 'homicidio'})
delitos_vida_lper['DELITO'] = delitos_vida_lper['DELITO'].replace({'LESIONES PERSONALES': 'lesiones personales'})
delitos_libertad_secu['DELITO'] = delitos_libertad_secu['DELITO'].replace({'SECUESTRO': 'Secuestro'})
delitos_patrimonio_hper['DELITO'] = delitos_patrimonio_hper['DELITO'].replace({'HURTO A PERSONAS': 'hurto a personas'})
delitos_patrimonio_hcom['DELITO'] = delitos_patrimonio_hcom['DELITO'].replace({'HURTO COMERCIO': 'hurto a comercio'})
delitos_patrimonio_hres['DELITO'] = delitos_patrimonio_hres['DELITO'].replace({'HURTO RESIDENCIAS': 'hurto a residencias'})
delitos_patrimonio_hveh['DELITO'] = delitos_patrimonio_hveh['DELITO'].replace({'HURTO VEHICULOS': 'hurto a vehículos'})
delitos_patrimonio_exto['DELITO'] = delitos_patrimonio_exto['DELITO'].replace({'EXTORSION': 'extorsión'})
delitos_familia_vif['DELITO'] = delitos_familia_vif['DELITO'].replace({'VIOLENCIA INTRAFAMILIAR': 'violencia intrafamiliar'})
delitos_seg_publica_terr['DELITO'] = delitos_seg_publica_terr['DELITO'].replace({'TERRORISMO': 'terrorismo'})


EJECUCION DEL CODIGO

In [73]:
from docx import Document
from docx.shared import Inches

# Para crear el documento en word
doc = Document()



doc.add_heading(f'Caracterización de la situación de seguridad en el municipio de {nombre_mpio} del departamento de {nombre_dpto}', 0)

doc.add_paragraph(
    f"Este documento presenta un análisis de los delitos registrados en el municipio de {nombre_mpio} del departamento de {nombre_dpto}. La información es tomada del Observatorio de Derechos Humanos y Defensa Nacional del Ministerio de Defensa Nacional. La información aquí presentada está sujeta a verificación de la fuente."
)


doc.add_heading(f'Año {segundo_año} comparado con {primer_año} ', 1)

#-------------------------------------------------------Año Corrido-------------------------------------------------------


# Resultados
doc.add_heading('Delitos contra la vida y la integridad personal', level=2)


# Verificar si el dataframe contiene los registros suficientes
if len(delitos_vida_hom_total) > 1:
    row_hom_total = delitos_vida_hom_total.iloc[1]

    if pd.isna(row_hom_total['VAR%']):
        variacion_texto_hom_total = "No hay datos suficientes para calcular la variación porcentual del año anterior."
    elif row_hom_total['VAR%'] > 0:
        variacion_texto_hom_total = f"un aumento del {row_hom_total['VAR%']:.2f}% con respecto al año anterior"
    else:
        variacion_texto_hom_total = f"una disminución del {abs(row_hom_total['VAR%']):.2f}% con respecto al año anterior" 
    
    # Texto del párrafo
    doc.add_paragraph(f"Los casos de {delitos_vida_hom_total['DELITO'].iloc[1]} de {delitos_vida_hom_total['AÑO'].iloc[1]}, presentaron {variacion_texto_hom_total} al pasar de {delitos_vida_hom_total['CANTIDAD'].iloc[0]} a "
                      f"{delitos_vida_hom_total['CANTIDAD'].iloc[1]} incidentes")
else:
    doc.add_paragraph(f"No se registraron casos de homicidios en el periodo de análisis.")



# Verificar si el dataframe contiene los registros suficientes
if len(delitos_vida_lper_total) > 1:
    try:
        row_lper_total = delitos_vida_lper_total.iloc[1]

        if pd.isna(row_lper_total['VAR%']):
            variacion_texto_lper_total = "No hay datos suficientes para calcular la variación porcentual del año anterior."
        elif row_lper_total['VAR%'] > 0:
            variacion_texto_lper_total = f"un aumento del {row_lper_total['VAR%']:.2f}% con respecto al año anterior"
        else:
            variacion_texto_lper_total = f"una disminución del {abs(row_lper_total['VAR%']):.2f}% con respecto al año anterior" 
        
    # Texto del párrafo
        doc.add_paragraph(f"Los casos de {delitos_vida_lper_total['DELITO'].iloc[1]} de {delitos_vida_lper_total['AÑO'].iloc[1]}, presentaron {variacion_texto_lper_total} al pasar de {delitos_vida_lper_total['CANTIDAD'].iloc[0]} a "
                        f"{delitos_vida_lper_total['CANTIDAD'].iloc[1]} incidentes")
    except Exception as e:
        # Manejar otros errores inesperados (p.ej., columnas faltantes)
        doc.add_paragraph(f"Error al procesar los datos: {str(e)}")
else:
    # Maneja el caso en que no hay suficientes filas
    doc.add_paragraph(f"No se registraron casos de lesiones personales en el periodo de análisis.")


doc.add_heading('Delitos contra la libertad individual y otras garantías constitucionales', level=2)


import pandas as pd

# Verificar si el dataframe contiene los registros suficientes
if len(delitos_libertad_secu_total) > 1:
    try:   
        row_secu_total = delitos_libertad_secu_total.iloc[1]

        if pd.isna(row_secu_total['VAR%']):
            variacion_texto_secu_total = "No hay datos suficientes para calcular la variación porcentual del año anterior."
        elif row_secu_total['VAR%'] > 0:
            variacion_texto_secu_total = f"un aumento del {row_secu_total['VAR%']:.2f}% con respecto al año anterior"
        else:
            variacion_texto_secu_total = f"una disminución del {abs(row_secu_total['VAR%']):.2f}% con respecto al año anterior" 
        
        # Texto del párrafo
        doc.add_paragraph(f"Los casos de {delitos_libertad_secu_total['DELITO'].iloc[1]} de {delitos_libertad_secu_total['AÑO'].iloc[1]}, presentaron {variacion_texto_secu_total} al pasar de {delitos_libertad_secu_total['CANTIDAD'].iloc[0]} a "
                        f"{delitos_libertad_secu_total['CANTIDAD'].iloc[1]} incidentes")
    except Exception as e:
        # Manejar otros errores inesperados (p.ej., columnas faltantes)
        doc.add_paragraph(f"Error al procesar los datos: {str(e)}")
else:
    # Maneja el caso en que no hay suficientes filas
    doc.add_paragraph(f"No se registraron casos de secuestro en el periodo de análisis.")


doc.add_heading('Delitos contra el patrimonio económico', level=2)

# Verificar si el dataframe contiene los registros suficientes
if len(delitos_patrimonio_hper_total) > 1:
    row_hper_total = delitos_patrimonio_hper_total.iloc[1]

    if pd.isna(row_hper_total['VAR%']):
        variacion_texto_hper_total = "No hay datos suficientes para calcular la variación porcentual del año anterior."
    elif row_hper_total['VAR%'] > 0:
        variacion_texto_hper_total = f"un aumento del {row_hper_total['VAR%']:.2f}% con respecto al año anterior"
    else:
        variacion_texto_hper_total = f"una disminución del {abs(row_hper_total['VAR%']):.2f}% con respecto al año anterior" 
    
    # Texto del párrafo
    doc.add_paragraph(f"Los casos de {delitos_patrimonio_hper_total['DELITO'].iloc[1]} de {delitos_patrimonio_hper_total['AÑO'].iloc[1]}, presentaron {variacion_texto_hper_total} al pasar de {delitos_patrimonio_hper_total['CANTIDAD'].iloc[0]} a "
                      f"{delitos_patrimonio_hper_total['CANTIDAD'].iloc[1]} incidentes")
else:
    # Maneja el caso en que no hay suficientes filas
    doc.add_paragraph(f"No se registraron casos de hurto a personas en el periodo de análisis.")


# Verificar si el dataframe contiene los registros suficientes
if len(delitos_patrimonio_hcom_total) > 1:
    row_hcom_total = delitos_patrimonio_hcom_total.iloc[1]

    if pd.isna(row_hcom_total['VAR%']):
        variacion_texto_hcom_total = "No hay datos suficientes para calcular la variación porcentual del año anterior."
    elif row_hcom_total['VAR%'] > 0:
        variacion_texto_hcom_total = f"un aumento del {row_hcom_total['VAR%']:.2f}% con respecto al año anterior"
    else:
        variacion_texto_hcom_total = f"una disminución del {abs(row_hcom_total['VAR%']):.2f}% con respecto al año anterior" 
    
    # Texto del párrafo
    doc.add_paragraph(f"Los casos de {delitos_patrimonio_hcom_total['DELITO'].iloc[1]} de {delitos_patrimonio_hcom_total['AÑO'].iloc[1]}, presentaron {variacion_texto_hcom_total} al pasar de {delitos_patrimonio_hcom_total['CANTIDAD'].iloc[0]} a "
                      f"{delitos_patrimonio_hcom_total['CANTIDAD'].iloc[1]} incidentes")
else:
    # Maneja el caso en que no hay suficientes filas
    doc.add_paragraph(f"No se registraron casos de hurto a comercio en el periodo de análisis.")


# Verificar si el dataframe contiene los registros suficientes
if len(delitos_patrimonio_hres_total) > 1:
    row_hres_total = delitos_patrimonio_hres_total.iloc[1]

    if pd.isna(row_hres_total['VAR%']):
        variacion_texto_hres_total = "No hay datos suficientes para calcular la variación porcentual del año anterior."
    elif row_hres_total['VAR%'] > 0:
        variacion_texto_hres_total = f"un aumento del {row_hres_total['VAR%']:.2f}% con respecto al año anterior"
    else:
        variacion_texto_hres_total = f"una disminución del {abs(row_hres_total['VAR%']):.2f}% con respecto al año anterior" 
    
    # Texto del párrafo
    doc.add_paragraph(f"Los casos de {delitos_patrimonio_hres_total['DELITO'].iloc[1]} de {delitos_patrimonio_hres_total['AÑO'].iloc[1]}, presentaron {variacion_texto_hres_total} al pasar de {delitos_patrimonio_hres_total['CANTIDAD'].iloc[0]} a "
                      f"{delitos_patrimonio_hres_total['CANTIDAD'].iloc[1]} incidentes")
else:
    # Maneja el caso en que no hay suficientes filas
    doc.add_paragraph(f"No se registraron casos de hurto a residencias en el periodo de análisis.")


# Verificar si el dataframe contiene los registros suficientes
if len(delitos_patrimonio_hveh_total) > 1:
    row_hveh_total = delitos_patrimonio_hveh_total.iloc[1]

    if pd.isna(row_hveh_total['VAR%']):
        variacion_texto_hveh_total = "No hay datos suficientes para calcular la variación porcentual del año anterior."
    elif row_hveh_total['VAR%'] > 0:
        variacion_texto_hveh_total = f"un aumento del {row_hveh_total['VAR%']:.2f}% con respecto al año anterior"
    else:
        variacion_texto_hveh_total = f"una disminución del {abs(row_hveh_total['VAR%']):.2f}% con respecto al año anterior" 
    
    # Texto del párrafo
    doc.add_paragraph(f"Los casos de {delitos_patrimonio_hveh_total['DELITO'].iloc[1]} de {delitos_patrimonio_hveh_total['AÑO'].iloc[1]}, presentaron {variacion_texto_hveh_total} al pasar de {delitos_patrimonio_hveh_total['CANTIDAD'].iloc[0]} a "
                      f"{delitos_patrimonio_hveh_total['CANTIDAD'].iloc[1]} incidentes")
else:
    # Maneja el caso en que no hay suficientes filas
    doc.add_paragraph(f"No se registraron casos de hurto a vehículos en el periodo de análisis.")


# Verificar si el dataframe contiene los registros suficientes
if len(delitos_patrimonio_exto_total) > 1:
    row_exto_total = delitos_patrimonio_exto_total.iloc[1]

    if pd.isna(row_exto_total['VAR%']):
        variacion_texto_exto_total = "No hay datos suficientes para calcular la variación porcentual del año anterior."
    elif row_exto_total['VAR%'] > 0:
        variacion_texto_exto_total = f"un aumento del {row_exto_total['VAR%']:.2f}% con respecto al año anterior"
    else:
        variacion_texto_exto_total = f"una disminución del {abs(row_exto_total['VAR%']):.2f}% con respecto al año anterior" 
    
    # Texto del párrafo
    doc.add_paragraph(f"Los casos de {delitos_patrimonio_exto_total['DELITO'].iloc[1]} de {delitos_patrimonio_exto_total['AÑO'].iloc[1]}, presentaron {variacion_texto_exto_total} al pasar de {delitos_patrimonio_exto_total['CANTIDAD'].iloc[0]} a "
                      f"{delitos_patrimonio_exto_total['CANTIDAD'].iloc[1]} incidentes")
else:
    # Maneja el caso en que no hay suficientes filas
    doc.add_paragraph(f"No se registraron casos de extorsión en el periodo de análisis.")

doc.add_heading('Delitos contra la familia', level=2)



# Verificar si el dataframe contiene los registros suficientes
if len(delitos_familia_vif_total) > 1:
    row_vif_total = delitos_familia_vif_total.iloc[1]

    if pd.isna(row_vif_total['VAR%']):
        variacion_texto_vif_total = "No hay datos suficientes para calcular la variación porcentual del año anterior."
    elif row_vif_total['VAR%'] > 0:
        variacion_texto_vif_total = f"un aumento del {row_vif_total['VAR%']:.2f}% con respecto al año anterior"
    else:
        variacion_texto_vif_total = f"una disminución del {abs(row_vif_total['VAR%']):.2f}% con respecto al año anterior" 
    
    # Texto del párrafo
    doc.add_paragraph(f"Los casos de {delitos_familia_vif_total['DELITO'].iloc[1]} de {delitos_familia_vif_total['AÑO'].iloc[1]}, presentaron {variacion_texto_vif_total} al pasar de {delitos_familia_vif_total['CANTIDAD'].iloc[0]} a "
                      f"{delitos_familia_vif_total['CANTIDAD'].iloc[1]} incidentes")
else:
    # Maneja el caso en que no hay suficientes filas
    doc.add_paragraph(f"No se registraron casos de violencia intrafamiliar en el periodo de análisis.")

doc.add_heading('Delitos contra la seguridad pública', level=2)

# Verificar si el dataframe contiene los registros suficientes
if len(delitos_seg_publica_terr_total) > 1:
    row_terr_total = delitos_seg_publica_terr_total.iloc[1]

    if pd.isna(row_terr_total['VAR%']):
        variacion_texto_terr = "No hay datos suficientes para calcular la variación porcentual del año anterior."
    elif row_terr_total['VAR%'] > 0:
        variacion_texto_terr_total = f"un aumento del {row_terr_total['VAR%']:.2f}% con respecto al año anterior"
    else:
        variacion_texto_terr_total = f"una disminución del {abs(row_terr_total['VAR%']):.2f}% con respecto al año anterior" 
    
    # Texto del párrafo
    doc.add_paragraph(f"Los casos de {delitos_seg_publica_terr_total['DELITO'].iloc[1]} de {delitos_seg_publica_terr_total['AÑO'].iloc[1]}, presentaron {variacion_texto_terr_total} al pasar de {delitos_seg_publica_terr_total['CANTIDAD'].iloc[0]} a "
                      f"{delitos_seg_publica_terr_total['CANTIDAD'].iloc[1]} incidentes")
else:
    # Maneja el caso en que no hay suficientes filas
    doc.add_paragraph(f"No se registraron casos de terrorismo en el periodo de análisis.")




#------------------------------------------------------- PERIODO DE AÑO-------------------------------------------------------

doc.add_heading(f'Periodo {min_mes_nombre} a {max_mes_nombre} de {segundo_año_periodo} comparado con el mismo periodo de {primer_año_periodo}', 1)

# Resultados
doc.add_heading('Delitos contra la vida y la integridad personal', level=2)

row_hom = delitos_vida_hom.iloc[1]

if pd.isna(row_hom['VAR%']):
    variacion_texto_hom = "No hay datos suficientes para calcular la variación porcentual del año anterior."
elif row_hom['VAR%'] > 0:
    variacion_texto_hom = f"un aumento del {row_hom['VAR%']:.2f}% con respecto al mismo periodo del año anterior"
else:
    variacion_texto_hom = f"una disminución del {abs(row_hom['VAR%']):.2f}% con respecto al mismo periodo del año anterior" 
    
doc.add_paragraph(f"En el año {delitos_vida_hom['AÑO'].iloc[1]}, los casos de {delitos_vida_hom['DELITO'].iloc[1]} presentaron {variacion_texto_hom} al pasar de {delitos_vida_hom['CANTIDAD'].iloc[0]} incidentes a  "
                f"{delitos_vida_hom['CANTIDAD'].iloc[1]} durante el periodo de {min_mes_nombre} a {max_mes_nombre}.")


row_lper = delitos_vida_lper.iloc[1]

if pd.isna(row_lper['VAR%']):
    variacion_texto_lper = "No hay datos suficientes para calcular la variación porcentual del año anterior."
elif row_lper['VAR%'] > 0:
    variacion_texto_lper = f"un aumento del {row_lper['VAR%']:.2f}% con respecto al mismo periodo del año anterior"
else:
    variacion_texto_lper = f"una disminución del {abs(row_lper['VAR%']):.2f}% con respecto al mismo periodo del año anterior" 
    
doc.add_paragraph(f"En el año {delitos_vida_lper['AÑO'].iloc[1]}, los casos de {delitos_vida_lper['DELITO'].iloc[1]} presentaron {variacion_texto_lper} al pasar de {delitos_vida_lper['CANTIDAD'].iloc[0]} incidentes a  "
                f"{delitos_vida_lper['CANTIDAD'].iloc[1]} durante el periodo de {min_mes_nombre} a {max_mes_nombre}.")


doc.add_heading('Delitos contra la libertad individual y otras garantías constitucionales', level=2)


import pandas as pd

# Verificar si el dataframe contiene los registros suficientes
if len(delitos_libertad_secu) > 1:
    row_secu = delitos_libertad_secu.iloc[1]

    if pd.isna(row_secu['VAR%']):
        variacion_texto_secu = "No hay datos suficientes para calcular la variación porcentual del año anterior."
    elif row_secu['VAR%'] > 0:
        variacion_texto_secu = f"un aumento del {row_secu['VAR%']:.2f}% con respecto al mismo periodo del año anterior"
    else:
        variacion_texto_secu = f"una disminución del {abs(row_secu['VAR%']):.2f}% con respecto al mismo periodo del año anterior" 
    
    # Texto del párrafo
    doc.add_paragraph(f"En el año {delitos_libertad_secu['AÑO'].iloc[1]}, los casos de {delitos_libertad_secu['DELITO'].iloc[1]} presentaron {variacion_texto_secu} al pasar de {delitos_libertad_secu['CANTIDAD'].iloc[0]} incidentes a "
                      f"{delitos_libertad_secu['CANTIDAD'].iloc[1]} durante el periodo de {min_mes_nombre} a {max_mes_nombre}.")
else:
    # Maneja el caso en que no hay suficientes filas
    doc.add_paragraph("No hay datos suficientes sobre delitos contra la libertad individual y otras garantías constitucionales para este período.")


doc.add_heading('Delitos contra el patrimonio económico', level=2)

# Verificar si el dataframe contiene los registros suficientes
if len(delitos_patrimonio_hper) > 1:
    row_hper = delitos_patrimonio_hper.iloc[1]

    if pd.isna(row_hper['VAR%']):
        variacion_texto_hper = "No hay datos suficientes para calcular la variación porcentual del año anterior."
    elif row_hper['VAR%'] > 0:
        variacion_texto_hper = f"un aumento del {row_hper['VAR%']:.2f}% con respecto al mismo periodo del año anterior"
    else:
        variacion_texto_hper = f"una disminución del {abs(row_hper['VAR%']):.2f}% con respecto al mismo periodo del año anterior" 
    
    # Texto del párrafo
    doc.add_paragraph(f"En el año {delitos_patrimonio_hper['AÑO'].iloc[1]}, los casos de {delitos_patrimonio_hper['DELITO'].iloc[1]} presentaron {variacion_texto_hper} al pasar de {delitos_patrimonio_hper['CANTIDAD'].iloc[0]} incidentes a "
                      f"{delitos_patrimonio_hper['CANTIDAD'].iloc[1]} durante el periodo de {min_mes_nombre} a {max_mes_nombre}.")
else:
    # Maneja el caso en que no hay suficientes filas
    doc.add_paragraph("No hay datos suficientes sobre delitos contra el patrimonio económico para este período.")


# Verificar si el dataframe contiene los registros suficientes
if len(delitos_patrimonio_hcom) > 1:
    row_hcom = delitos_patrimonio_hcom.iloc[1]

    if pd.isna(row_hcom['VAR%']):
        variacion_texto_hcom = "No hay datos suficientes para calcular la variación porcentual del año anterior."
    elif row_hcom['VAR%'] > 0:
        variacion_texto_hcom = f"un aumento del {row_hcom['VAR%']:.2f}% con respecto al mismo periodo del año anterior"
    else:
        variacion_texto_hcom = f"una disminución del {abs(row_hcom['VAR%']):.2f}% con respecto al mismo periodo del año anterior" 
    
    # Texto del párrafo
    doc.add_paragraph(f"En el año {delitos_patrimonio_hcom['AÑO'].iloc[1]}, los casos de {delitos_patrimonio_hcom['DELITO'].iloc[1]} presentaron {variacion_texto_hcom} al pasar de {delitos_patrimonio_hcom['CANTIDAD'].iloc[0]} incidentes a "
                      f"{delitos_patrimonio_hcom['CANTIDAD'].iloc[1]} durante el periodo de {min_mes_nombre} a {max_mes_nombre}.")
else:
    # Maneja el caso en que no hay suficientes filas
    doc.add_paragraph("No hay datos suficientes sobre delitos contra el patrimonio económico para este período.")


# Verificar si el dataframe contiene los registros suficientes
if len(delitos_patrimonio_hres) > 1:
    row_hres = delitos_patrimonio_hres.iloc[1]

    if pd.isna(row_hres['VAR%']):
        variacion_texto_hres = "No hay datos suficientes para calcular la variación porcentual del año anterior."
    elif row_hres['VAR%'] > 0:
        variacion_texto_hres = f"un aumento del {row_hres['VAR%']:.2f}% con respecto al mismo periodo del año anterior"
    else:
        variacion_texto_hres = f"una disminución del {abs(row_hres['VAR%']):.2f}% con respecto al mismo periodo del año anterior" 
    
    # Texto del párrafo
    doc.add_paragraph(f"En el año {delitos_patrimonio_hres['AÑO'].iloc[1]}, los casos de {delitos_patrimonio_hres['DELITO'].iloc[1]} presentaron {variacion_texto_hres} al pasar de {delitos_patrimonio_hres['CANTIDAD'].iloc[0]} incidentes a "
                      f"{delitos_patrimonio_hres['CANTIDAD'].iloc[1]} durante el periodo de {min_mes_nombre} a {max_mes_nombre}.")
else:
    # Maneja el caso en que no hay suficientes filas
    doc.add_paragraph("No hay datos suficientes sobre delitos contra el patrimonio económico para este período.")


# Verificar si el dataframe contiene los registros suficientes
if len(delitos_patrimonio_hveh) > 1:
    row_hveh = delitos_patrimonio_hveh.iloc[1]

    if pd.isna(row_hveh['VAR%']):
        variacion_texto_hveh = "No hay datos suficientes para calcular la variación porcentual del año anterior."
    elif row_hveh['VAR%'] > 0:
        variacion_texto_hveh = f"un aumento del {row_hveh['VAR%']:.2f}% con respecto al mismo periodo del año anterior"
    else:
        variacion_texto_hveh = f"una disminución del {abs(row_hveh['VAR%']):.2f}% con respecto al mismo periodo del año anterior" 
    
    # Texto del párrafo
    doc.add_paragraph(f"En el año {delitos_patrimonio_hveh['AÑO'].iloc[1]}, los casos de {delitos_patrimonio_hveh['DELITO'].iloc[1]} presentaron {variacion_texto_hveh} al pasar de {delitos_patrimonio_hveh['CANTIDAD'].iloc[0]} incidentes a "
                      f"{delitos_patrimonio_hveh['CANTIDAD'].iloc[1]} durante el periodo de {min_mes_nombre} a {max_mes_nombre}.")
else:
    # Maneja el caso en que no hay suficientes filas
    doc.add_paragraph("No hay datos suficientes sobre delitos contra el patrimonio económico para este período.")


# Verificar si el dataframe contiene los registros suficientes
if len(delitos_patrimonio_exto) > 1:
    row_exto = delitos_patrimonio_exto.iloc[1]

    if pd.isna(row_exto['VAR%']):
        variacion_texto_exto = "No hay datos suficientes para calcular la variación porcentual del año anterior."
    elif row_exto['VAR%'] > 0:
        variacion_texto_exto = f"un aumento del {row_exto['VAR%']:.2f}% con respecto al mismo periodo del año anterior"
    else:
        variacion_texto_exto = f"una disminución del {abs(row_exto['VAR%']):.2f}% con respecto al mismo periodo del año anterior" 
    
    # Texto del párrafo
    doc.add_paragraph(f"En el año {delitos_patrimonio_exto['AÑO'].iloc[1]}, los casos de {delitos_patrimonio_exto['DELITO'].iloc[1]} presentaron {variacion_texto_exto} al pasar de {delitos_patrimonio_exto['CANTIDAD'].iloc[0]} incidentes a "
                      f"{delitos_patrimonio_exto['CANTIDAD'].iloc[1]} durante el periodo de {min_mes_nombre} a {max_mes_nombre}.")
else:
    # Maneja el caso en que no hay suficientes filas
    doc.add_paragraph("No hay datos suficientes sobre delitos contra el patrimonio económico para este período.")

doc.add_heading('Delitos contra la familia', level=2)



# Verificar si el dataframe contiene los registros suficientes
if len(delitos_familia_vif) > 1:
    row_vif = delitos_familia_vif.iloc[1]

    if pd.isna(row_vif['VAR%']):
        variacion_texto_vif = "No hay datos suficientes para calcular la variación porcentual del año anterior."
    elif row_vif['VAR%'] > 0:
        variacion_texto_vif = f"un aumento del {row_vif['VAR%']:.2f}% con respecto al mismo periodo del año anterior"
    else:
        variacion_texto_vif = f"una disminución del {abs(row_vif['VAR%']):.2f}% con respecto al mismo periodo del año anterior" 
    
    # Tvif del párrafo
    doc.add_paragraph(f"En el año {delitos_familia_vif['AÑO'].iloc[1]}, los casos de {delitos_familia_vif['DELITO'].iloc[1]} presentaron {variacion_texto_vif} al pasar de {delitos_familia_vif['CANTIDAD'].iloc[0]} incidentes a "
                      f"{delitos_familia_vif['CANTIDAD'].iloc[1]} durante el periodo de {min_mes_nombre} a {max_mes_nombre}.")
else:
    # Maneja el caso en que no hay suficientes filas
    doc.add_paragraph("No hay datos suficientes sobre delitos contra la familia para este período.")

doc.add_heading('Delitos contra la seguridad pública', level=2)

# Verificar si el dataframe contiene los registros suficientes
if len(delitos_seg_publica_terr) > 1:
    row_terr = delitos_seg_publica_terr.iloc[1]

    if pd.isna(row_terr['VAR%']):
        variacion_texto_terr = "No hay datos suficientes para calcular la variación porcentual del año anterior."
    elif row_terr['VAR%'] > 0:
        variacion_texto_terr = f"un aumento del {row_terr['VAR%']:.2f}% con respecto al mismo periodo del año anterior"
    else:
        variacion_texto_terr = f"una disminución del {abs(row_terr['VAR%']):.2f}% con respecto al mismo periodo del año anterior" 
    
    # Tterr del párrafo
    doc.add_paragraph(f"En el año {delitos_seg_publica_terr['AÑO'].iloc[1]}, los casos de {delitos_seg_publica_terr['DELITO'].iloc[1]} presentaron {variacion_texto_terr} al pasar de {delitos_seg_publica_terr['CANTIDAD'].iloc[0]} incidentes a "
                      f"{delitos_seg_publica_terr['CANTIDAD'].iloc[1]} durante el periodo de {min_mes_nombre} a {max_mes_nombre}.")
else:
    # Maneja el caso en que no hay suficientes filas
    doc.add_paragraph("No hay datos suficientes sobre delitos contra la seguridad pública para este período.")


# Guardar el documento
file_path = f'C:/Users/USUARIO/OneDrive - Departamento Nacional de Planeacion/maalarcon_files/DNP-583-2024/240411 Bullets Ibagué/{codigo_dane}_{nombre_mpio}_analisis_delitos2.docx'
doc.save(file_path)

file_path


'C:/Users/USUARIO/OneDrive - Departamento Nacional de Planeacion/maalarcon_files/DNP-583-2024/240411 Bullets Ibagué/73001_Ibagué_analisis_delitos2.docx'